В базе данных компании присутствуют следующие таблицы:

`city`<br><br>
city_id – id города, где проходит акция<br>
client_city_id – id города, где находится покупатель<br>
city – город<br>
client_city – город, где находится покупатель <br>

`client`<br><br>
client_id – id покупателя<br>
client_city_id – id города, где находится покупатель<br>
birth_date – дата рождения покупателя<br>
registration – дата регистрации покупателя<br>

`promotion`<br><br>
promotion_id – id акции<br>
category_id – id категории<br>
promotion_name – акция<br>
category_name – категория<br>
partner_id – id партнёра<br>
partner_name – партнёр<br>

`purchase`<br><br>
purchase_id – id покупки<br>
partner_id – id партнёра<br>
client_id – id покупателя<br>
city_id – id города<br>
promotion_id – id акции<br>
category_id – id категории<br>
purchase_date – дата покупки<br>
price – цена за единицу товара<br>
quantity – число проданных единиц<br>
status – статус покупки<br>

Нужно написать запрос, чтобы получить такую таблицу для поиска целевой аудитории и лояльных клиентов<br>

purchase_date – дата покупки<br>
purchase_id – id покупки<br>
client_id – id покупателя<br>
client_age – возраст покупателя<br>
client_registration_age – как долго человек пользуется вашими магазинами<br>
client_category – содержит new или old, если это первая или последующая покупка соответственно<br>
promotion_name – акция<br>
category_name – категория<br>
partner_name – партнёр<br>
client_city – город, где находится покупатель<br>
city – город<br>
revenue – сумма выручки<br>
quantity – число проданных единиц<br><br>
При этом в таблице должны присутствовать только значения, где в поле status стоит значение 1, и только те, где purchase_date находится в диапазоне от 01.05.2020 до 01.08.2020

In [1]:
import pandas as pd
import pandahouse as ph

In [15]:
# Объявляем параметры подключения к clickhouse

connection = {'host': 'http://clickhouse.beslan.pro:8080',
              'database':'project_variant_2',
              'user':'student',
              'password':'dpo_python_2020'}

In [16]:
query = """
SELECT 
    --нужные данные из таблицы purchase
    purchase_date, purchase_id, p.client_id AS client_id, 
    
    --возраст покупателя (в годах):
    dateDiff('year',c.birth_date, now()) AS client_age, 
    
    --как долго покупатель пользуется магазинами (в месяцах):
    dateDiff('month',c.registration, now()) AS client_registration_age,
    
    -- столбец client_category - присваиваем операции new|old в зависимости от того, новая или последующая эта покупка:
    IF (purchase_id IN 
    -- список purchase_id всех первых покупок всех клиентов:
        (
        --(Некоторые клиенты в первый день покупок совершили несколько операций - несколько покупок под разными purchase_id.
        -- В таких случаях первой покупкой (new) будет считаться та, у которой наименьший purchase_id)
        SELECT MIN(purchase_id) 
        FROM {db}.purchase AS p
        JOIN
            --таблица с датами первых покупок для каждого клиента:
            (SELECT client_id, MIN(purchase_date) AS first_date FROM {db}.purchase 
            GROUP BY client_id
            ) AS r
        ON p.client_id=r.client_id AND p.purchase_date=r.first_date
        GROUP BY  client_id, purchase_date
        ),
        'new','old' ) AS client_category,
    
    --нужные данные из таблицы promotion:
    pr.promotion_name AS promotion_name, pr.category_name AS category_name, pr.partner_name AS partner_name, 
    
    --нужные данные из таблицы city:
    c2.city AS client_city, c1.city AS city,
    
    --нужные данные из таблицы purchase:
    toUInt64(p.price*p.quantity) AS revenue, toUInt64(p.quantity) AS quantity

FROM {db}.purchase AS p

--джойним с таблицами client и promotion для получения из них нужных данных:
LEFT JOIN {db}.client AS c ON p.client_id = c.client_id
LEFT JOIN {db}.promotion AS pr ON p.partner_id = pr.partner_id

-- В таблице city каждому городу (Москва, Санкт-Петербург, Новгород,..) присвоен id от 0 до 8 и перечислены возможные 
-- комбинации этих городов. Таких комбинаций должно быть 9*9= 81, но строк в таблице всего 77, т.е. некоторые 
-- комбинации "город - город, где находится покупатель" отсутствуют (н-р, "Щучинщина - Щучинщина"), хотя на деле 
-- операции для таких комбинаций в таблице purchase есть. Поэтому, чтобы не потерять данные, будем двойнить с таблицей,
-- в которой перечислены id и соответсвующие им города:
LEFT JOIN (SELECT DISTINCT city_id, city FROM {db}.city) AS c1 ON p.city_id = c1.city_id
LEFT JOIN (SELECT DISTINCT city_id, city FROM {db}.city) AS c2 ON c.client_city_id = c2.city_id


--фильтруем данные:
WHERE status = 1 AND purchase_date BETWEEN '2020-05-01' AND '2020-08-01'

"""

In [17]:
data = ph.read_clickhouse(query, connection=connection)
data

,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-06-05,0,853,29,22,old,campaign_3,Овощи,Всем партнёрам партнёр,Ярославль,Киев,150,5
1,2020-06-15,1,406,24,18,old,campaign_3,Овощи,Всем партнёрам партнёр,Минск,Новгород,1150,5
2,2020-08-01,9,867,25,10,old,campaign_3,Овощи,Всем партнёрам партнёр,Киев,Москва,1840,4
3,2020-07-16,15,109,30,29,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Щучинщина,Санкт-Петербург,440,8
4,2020-06-29,22,528,21,12,old,campaign_1,Бакалея,Google,Щучинщина,Щучинщина,390,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730,2020-07-28,8241,602,23,17,old,campaign_1,Бакалея,Google,Щучинщина,Вена,780,6
2731,2020-07-15,8245,46,31,27,old,campaign_3,Овощи,Всем партнёрам партнёр,Москва,Минск,2275,5
2732,2020-06-09,8246,954,25,13,old,campaign_3,Овощи,Всем партнёрам партнёр,Санкт-Петербург,Вена,3120,8
2733,2020-08-01,8247,279,22,17,old,campaign_1,Бакалея,Google,Казань,Новгород,735,7


In [10]:
# Если хотим выгрузить в эксель файл:
# data.to_excel("./Data.xlsx", sheet_name='Data', index=False)